In [3]:
import pandas as pd
from geopy.geocoders import Nominatim

In [4]:
InitialDF = pd.read_csv(r"C:\Users\miche\programs\ProgWeather\progetTempo\data\GlobalLandTemperaturesByMajorCity.csv")
InitialDF['dt'] = pd.to_datetime(InitialDF['dt'])

geo = Nominatim(user_agent='MapApp')

In [5]:
def latitude(x):
    if (x[len(x) - 1] == 'N'):
        i = float(x[:len(x) - 1])
    else:
        i = -float(x[:len(x) - 1])
    return i


def longitude(x):
    if (x[len(x) - 1] == 'E'):
        i = float(x[:len(x) - 1])
    else:
        i = -float(x[:len(x) - 1])
    return i

In [20]:
def coordinates(city):
    return [latitude(city['Latitude']), longitude(city['Longitude'])]

citiesLonLat = {}

for i in range(len(InitialDF)):
    try:
        if InitialDF['City'][i] in citiesLonLat:
            continue
        info = geo.geocode(InitialDF['City'][i] + " " + InitialDF['Country'][i])
        citiesLonLat[InitialDF['City'][i]] = [info.latitude, info.longitude] 
    except:
        pass
for i in range(len(InitialDF)):
    if not InitialDF['City'][i] in citiesLonLat:
        citiesLonLat[InitialDF['City'][i]] = coordinates(cities.iloc[i])
np.save("coordinates_correct", citiesLonLat)


In [25]:
corr_coord = np.load("coordinates_correct.npy", allow_pickle=True).item()

data_list = [{'City': city, 'Lat': corr_coord[0], 'Lon': corr_coord[1]} for city, corr_coord in corr_coord.items()]

unique_cities = pd.DataFrame(data_list)
unique_cities

,City,Lat,Lon
0,Abidjan,5.320357,-4.016107
1,Addis Abeba,9.035829,38.752413
2,Ahmadabad,23.021624,72.579707
3,Aleppo,36.199240,37.163725
4,Alexandria,31.199181,29.895172
...,...,...,...
95,Tokyo,35.684057,139.774491
96,Toronto,43.653482,-79.383935
97,Umm Durman,15.644554,32.477731
98,Wuhan,30.595105,114.299935


In [10]:
unique_cities = InitialDF['City'].unique().tolist()

cities = []
latitudes = []
longitudes = []

for city in unique_cities:
    cities.append(city)
    row=InitialDF[InitialDF['City']==city].iloc[0]
    latitudes.append(latitude(row['Latitude']))
    longitudes.append(longitude(row['Longitude'])) 
    
coordinates_df = pd.DataFrame({
    'City': cities,
    'Latitude': latitudes,
    'Longitude': longitudes
})

# Print the resulting DataFrame
coordinates_df.head()

,City,Latitude,Longitude
0,Abidjan,5.63,-3.23
1,Addis Abeba,8.84,38.11
2,Ahmadabad,23.31,72.52
3,Aleppo,36.17,37.79
4,Alexandria,31.35,30.16


#### Manually modyfing the coordinates of three cities because for some reason the wrong coordinates were provided


In [29]:

unique_cities[unique_cities["City"]== "Brasília"]  = ["Brasília", -15.7942287, -47.8821658]
unique_cities[unique_cities["City"]== "Kinshasa"] = ['Kinshasa', -4.325, 15.322222]
unique_cities[unique_cities["City"]== "Izmir"] = ['Izmir', 38.423733, 27.142826]
unique_cities.to_csv("Unique_cities_coordinates.csv")
unique_cities.head()

,City,Lat,Lon
0,Abidjan,5.320357,-4.016107
1,Addis Abeba,9.035829,38.752413
2,Ahmadabad,23.021624,72.579707
3,Aleppo,36.199240,37.163725
4,Alexandria,31.199181,29.895172


In [ ]:
#### Obtaining a final Dataframe with the desired columns, and saving it to a csv for more convenient access

In [30]:
finalDFONE = pd.merge( unique_cities,InitialDF, on='City', how='inner')
finalDFONE = finalDFONE[['City','Lat', 'Lon','AverageTemperature','dt']]
final = pd.DataFrame(finalDFONE)
final.rename(columns = {'Lat':'Latitude', 'Lon':'Longitude'}, inplace = True)
final["dt"] = pd.to_datetime(final["dt"])
final.head()
final.to_csv("DFtoImport.csv")